In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

importing ngsxfem-2.1.2405


In [2]:
maxh = 0.2
# Finite element space order
order = 2
# Stabilization parameter for ghost-penalty
gamma_stab = 100
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

r = sqrt(x**2 + y**2)
levelset = r-1

beta0 =1
beta1 = 1
beta2 = 1
beta3 = 1
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)
#cut_elements = ci.GetElementsOfType(HASIF)  


interior_facets = GetFacetsWithNeighborTypes(mesh, a=neg, b=neg)
interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)

h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)

dw = dFacetPatch(definedonelements=ba_facets)
dw_int = dFacetPatch(definedonelements=interior_facets)
dw_interface = dFacetPatch(definedonelements=interface_facet_set)


In [4]:
V = VectorH1(mesh, order=order,dgjumps=True)
V = Compress(V, GetDofsOfElements(V,hasneg))
Q = H1(mesh, order=order-1)
Q = Compress(Q, GetDofsOfElements(Q,hasneg))
Z = NumberSpace(mesh)
X = V*Q*Z
(u,p,z),(v,q,y) = X.TnT()
gfu = GridFunction(X)

def jump(f):
    return f - f.Other()
def jumpn(f):
    n = Normalize(grad(lsetp1))
    return Grad(f)*n - Grad(f).Other()*n

a = BilinearForm(X)
stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #nitzshe stabilization
stokes += -(q*n * u + p*n * v) * ds
stokes += p*y *dx + q *z*dx
stokes += beta2* InnerProduct(jump(Grad(u)), jump(Grad(v))) * dw_interface #velocity ghost penalty
stokes += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface #velocity ghost penalty
a += stokes 
a.Assemble()
#a.mat += a_vel.mat

f = CF((4*y,x))
rhs = LinearForm(X)  # oder f*v*dx mit f gegeben
rhs += InnerProduct(f, v)* dx
rhs.Assemble()

gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * rhs.vec

#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
#Draw(gfu.components[0], mesh, "u")
#Draw(gfu.components[1], mesh, "p")

DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
DrawDC(lsetp1,gfu.components[1], 0, mesh, "p", min = -0.1, max = 0.1, autoscale=False)


webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [5]:
sceneu =DrawDC(lsetp1,gfu.components[0], CF((0,0)), mesh, "u")
scenep = DrawDC(lsetp1,gfu.components[1], 0, mesh, "p", min = -0.1, max = 0.1, autoscale=False)

webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [6]:
dt = 0.001
print(type(stokes))
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)
mstar += 1/dt *InnerProduct(u,v)*dx + stokes
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs())

conv = BilinearForm(X, nonassemble = True)
conv += (Grad(u) * u) * v * dx



<class 'ngsolve.comp.SumOfIntegrals'>


In [7]:
tend = 0
gfut = GridFunction(gfu.space,multidim=0)

tend += 1
gfut.AddMultiDimComponent(gfu.vec)
t = 0; cnt = 0
while t < tend-0.5*dt:
    print ("\rt=", t, end="")

    conv.Assemble()
    res = (a.mat + conv.mat)* gfu.vec
    gfu.vec.data -=  inv * res

    t = t + dt; cnt += 1
    sceneu.Redraw()
    scenep.Redraw()
    if cnt % 50 == 0:
        gfut.AddMultiDimComponent(gfu.vec)

t= 0craete bilinearformapplication
t= 0.999000000000000876

In [8]:
DrawDC (lsetp1,gfut.components[0],CF((0,0)), mesh, interpolate_multidim=True, animate=True,
       vectors = True)
Draw (gfut.components[1], mesh, interpolate_multidim=True, animate=True,
      min=-0.3, max=0.3, autoscale=False)
DrawDC (lsetp1,gfut.components[1],0, mesh, min = -0.1, max = 0.1, autoscale=False ,interpolate_multidim=True, animate=True,
       vectors = True)

webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene